# Objective

1. Design a model for Mnist with 10 independent fully connected layers.
2. Add a average layer which averages the results of the 10 previous layers
3. Add a layer which generates the classification results with softmax.

# Prepare Environment

In [1]:
%env KERAS_BACKEND=tensorflow
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

env: KERAS_BACKEND=tensorflow


# Prepare Data
1. Load data

In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


2. Preprocess input
    - Without reshaping the input data from 28 x 28 to 784, which will be done by a flatten layer later

In [3]:
# normalization
x_train = x_train / 255
x_test = x_test / 255

print('x_train.shape =', x_train.shape)
print('x_test.shape =', x_test.shape)

x_train.shape = (60000, 28, 28)
x_test.shape = (10000, 28, 28)


3. Preprocess output

In [4]:
# categorical
import keras.utils as np_utils
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

print('y_train.shape =', y_train.shape)
print('y_test.shape =', y_test.shape)

y_train.shape = (60000, 10)
y_test.shape = (10000, 10)


# Build Model
1. Input layer
    - Add a flatten layer to tranfrom the two dimensional input (28 x 28) to one dimension (784)

In [5]:
from keras.layers import Input, Flatten

x = Input(shape=(28, 28))
flatten = Flatten()(x)

2. 10 independent full-connected layers

In [6]:
from keras.layers import Dense

dense_layers = []
for i in range(10):
    dense_layers.append(Dense(10, activation='relu'))

print(dense_layers)

[<keras.layers.core.Dense object at 0x00000239B076CB00>, <keras.layers.core.Dense object at 0x00000239B076CA58>, <keras.layers.core.Dense object at 0x00000239B09EF320>, <keras.layers.core.Dense object at 0x00000239B09EF390>, <keras.layers.core.Dense object at 0x00000239B09EF5F8>, <keras.layers.core.Dense object at 0x00000239B09EF780>, <keras.layers.core.Dense object at 0x00000239B09EF908>, <keras.layers.core.Dense object at 0x00000239B09EFA90>, <keras.layers.core.Dense object at 0x00000239B09EFC18>, <keras.layers.core.Dense object at 0x00000239B09EFDA0>]


In [7]:
h1 = []
for layer in dense_layers:
    h1.append(layer(flatten))

3. Average layer

In [8]:
import keras.backend as K

def average_function(inputs):
    return K.mean(inputs, axis=-1, keepdims=True)

In [9]:
from keras.layers import Lambda

average_layer = Lambda(average_function)
h2 = []
for h in h1:
    h2.append(average_layer(h))

4. Concatenate average results

In [10]:
from keras.layers import concatenate, Activation
u = concatenate(h2)
y = Activation('softmax')(u)

5. Create model

In [11]:
from keras.models import Model

model = Model(x, y)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28)       0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 784)          0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10)           7850        flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 10)           7850        flatten_1[0][0]                  
__________________________________________________________________________________________________
dense_3 (D

6. Train

In [12]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
60000/60000 [==============================] - 17s 278us/step - loss: 0.5134 - acc: 0.8780
Epoch 2/10
60000/60000 [==============================] - 12s 201us/step - loss: 0.2969 - acc: 0.9181
Epoch 3/10
60000/60000 [==============================] - 12s 201us/step - loss: 0.2641 - acc: 0.9259
Epoch 4/10
60000/60000 [==============================] - 12s 201us/step - loss: 0.2450 - acc: 0.9316
Epoch 5/10
60000/60000 [==============================] - 12s 201us/step - loss: 0.2306 - acc: 0.9348
Epoch 6/10
60000/60000 [==============================] - 12s 203us/step - loss: 0.2179 - acc: 0.9388
Epoch 7/10
60000/60000 [==============================] - 12s 203us/step - loss: 0.2070 - acc: 0.9413
Epoch 8/10
60000/60000 [==============================] - 12s 202us/step - loss: 0.1969 - acc: 0.9444
Epoch 9/10
60000/60000 [==============================] - 12s 203us/step - loss: 0.1881 - acc: 0.9474
Epoch 10/10
60000/60000 [==============================] - 12s 203us/step - loss: 

In [13]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 50us/step


[0.18592509019076825, 0.9453]